In [2]:
import tokenize
import pydriller as pyd
import pandas as pd
from datetime import datetime
import ast

In [ ]:
for commit in pyd.RepositoryMining(path_to_repo=path, since=tf_dt1, to=tf_dt2, only_authors=Authors).traverse_commits():
    for modified_file in commit.modifications:
        if modified_file.filename.endswith(".py"):
            tf_source = tf_source.append({'commit_ID': commit.hash, 'Author': commit.author.name,
                                          'msg': commit.msg,'Commit_before': modified_file.source_code_before,
                                          'Commit_after': modified_file.source_code}, ignore_index=True)
                #print("Code", commit.hash, commit.author.name, commit.msg,commit.author_date,
                        # modified_file.filename, flush=True)

In [1282]:
class FuncParser_onlycall(ast.NodeVisitor):
    def visit_Import(self, node): 
        ast.NodeVisitor.generic_visit(self, node)
    def visit_ImportFrom(self, node):
        if(node.module != None):
            n3= (node.module)
            file_contents.append([n3])
        ast.NodeVisitor.generic_visit(self, node)
    def visit_alias(self, node): 
        n1 =  (node.name)
        file_contents.append([n1])
        if(node.asname != None):
            n2 = (node.asname)
            file_contents.append([n2])
        ast.NodeVisitor.generic_visit(self, node)
    def visit_Call(self, node): 
        attrib = node.func
        if(attrib.__class__.__name__ == 'Attribute'):
            file_contents.append([attrib.attr])
    def generic_visit(self, node):
        ast.NodeVisitor.generic_visit(self, node)
    

In [1423]:
stop_word = {'ctx','value','name','values','kw_defaults','kwonlyargs','kwarg','n','body','args', 'defaults', 'vararg', 'arguments'}
class FuncParser(ast.NodeVisitor):
    def visit_Load(self, node): pass
    def visit_Store(self, node): pass
    def visit_Str(self, node): pass
    def visit_Num(self, node): pass
    def visit_Module(self, node): 
        ast.NodeVisitor.generic_visit(self, node)
    def visit_arguments(self, node): 
        ast.NodeVisitor.generic_visit(self, node)
    def visit_Expr(self, node): 
        ast.NodeVisitor.generic_visit(self, node)
    def visit_Assign(self, node): 
        ast.NodeVisitor.generic_visit(self, node)
    def visit_FunctionDef(self,node):
        file_contents.append([node.__class__.__name__])
        file_contents.append([node.name])
        ast.NodeVisitor.generic_visit(self, node)
    def visit_Call(self, node): 
        attrib = node.func
        if(attrib.__class__.__name__ == 'Attribute'):
            file_contents.append([attrib.attr])
        elif (attrib.__class__.__name__ == 'Name'):
            file_contents.append([attrib.id])
    def visit_arg(self, node): 
        ast.NodeVisitor.generic_visit(self, node)
    def visit_BinOp(self,node):
        temp = (node.op)
        op = "BinOp: "+temp.__class__.__name__ +"()"
        file_contents.append([op])
    def visit_Import(self, node): 
        ast.NodeVisitor.generic_visit(self, node)
    def visit_ImportFrom(self, node):
        if(node.module != None):
            n3= (node.module)
            file_contents.append([n3])
        ast.NodeVisitor.generic_visit(self, node)
    def visit_alias(self, node): 
        n1 =  (node.name)
        file_contents.append([n1])
        if(node.asname != None):
            n2 =  (node.asname)
            file_contents.append([n2])
        ast.NodeVisitor.generic_visit(self, node)
    def visit_Name(self, node):
       # temp ="Name:"+node.id
        #file_contents.append([node.id])
        ast.NodeVisitor.generic_visit(self, node)
    def visit_Attribute(self, node): 
        #temp1 ="Name:"+node.attr
        file_contents.append([node.attr])
        ast.NodeVisitor.generic_visit(self, node)
    def generic_visit(self, node):
        file_contents.append([node.__class__.__name__])
        #for f in node._fields:
           # if(not f in stop_word):     
              #  file_contents.append([f])
        #ast.iter_fields(node)
        ast.NodeVisitor.generic_visit(self, node)

In [119]:
#First we need to put the 'commit_before' and 'commit_after' into two different text file
#becuase AST accept a string which is a read line by line of a code file. but while we store the commit in a cell of dataframe, it is not able to be read line by line
text_before=tf_source['Commit_before'][1]
text_after=tf_source['Commit_after'][1]
before = open("before-commit.py", "w+")
after = open("after-commit.py", "w+")
before.write(text_before)
after.write(text_after)

226301

In [1424]:
with tokenize.open("co.py") as sf:  # need the tokenize.open for source files and not a string
    source_file_before = sf.read()

ast_before = ast.parse(source_file_before)
bf_obj = FuncParser()
bf_tree = ast.parse(ast_before)
file_contents = []
bf_obj.visit(bf_tree)

In [1289]:
for node in ast.walk(bf_tree):
    print(node.__class__.__name__)

Module
FunctionDef
arguments
Assign
If
Return
arg
Name
Num
BoolOp
Assign
If
Name
Store
Or
Name
Call
Name
Call
Compare
Assign
Load
Load
Name
Name
Name
Store
Name
Call
Name
Eq
Num
Name
Call
Load
Load
Load
Load
Attribute
Load
Store
Name
Name
Name
Load
Load
Load
Load


In [1308]:
print(ast.dump(bf_tree))

Module(body=[ImportFrom(module='sklearn.feature_extraction.text', names=[alias(name='TfidfTransformer', asname=None)], level=0), Assign(targets=[Name(id='X_train_tokens', ctx=Store())], value=Call(func=Attribute(value=Name(id='data', ctx=Load()), attr='drop', ctx=Load()), args=[List(elts=[Str(s='commit_Author'), Str(s='commit_msg')], ctx=Load())], keywords=[keyword(arg='axis', value=Num(n=1))])), Assign(targets=[Name(id='tfidf_transformer', ctx=Store())], value=Call(func=Name(id='TfidfTransformer', ctx=Load()), args=[], keywords=[keyword(arg='use_idf', value=NameConstant(value=True))])), Expr(value=Call(func=Attribute(value=Name(id='tfidf_transformer', ctx=Load()), attr='fit_transform', ctx=Load()), args=[Name(id='X_train_tokens', ctx=Load())], keywords=[]))])


In [1425]:
file_contents

[['FunctionDef'],
 ['dict_keys_to_ordered_list'],
 ['If'],
 ['BoolOp'],
 ['Or'],
 ['isinstance'],
 ['list'],
 ['If'],
 ['Compare'],
 ['Eq'],
 ['try_sort'],
 ['BinOp: Add()'],
 ['Return']]

In [16]:
with tokenize.open("after-commit.py") as sf:  # need the tokenize.open for source files and not a string
    source_file_after = sf.read()

after = ast.parse(source_file_after)
ast_after = ast.parse(source_file_after)
aft_obj = FuncParser()
aft_tree = ast.parse(ast_after)
file_contents = []
aft_obj.visit(aft_tree)

In [18]:
diff = tokens_after.subtract(tokens_before)

In [19]:
diff_token = diff[(diff.select_dtypes(include=['number']) != 0).any(1)]
diff_token=diff_token.fillna(0)
diff_token= diff_token.abs()
diff_token = diff_token.reset_index()
diff_token

,index,token


In [232]:
df_author_token = pd.DataFrame(columns=['Author','msg', 'tokens', 'counts'])

In [233]:
df_author_token = df_author_token.append({'Author':tf_source['Author'][1],'msg':tf_source['msg'][1]
                                         ,'tokens':diff_token['index'],'counts':diff_token['token']},ignore_index=True)

In [235]:
x=df_author_token['tokens'][0]
y=df_author_token['counts'][0]

In [236]:
vocab = {}
i = 1
for word in x:
    if word in vocab:
        continue
    else:
        vocab[word]=i
        i+=1
print(vocab)

{'Import:combinations': 1, 'Import:parameterized': 2, 'Import:test_util': 3, 'ImportFrom:absl.testing': 4}


In [237]:
one = [df_author_token['Author'][0]]+[0]*len(vocab)
one

['Allen Lavoie', 0, 0, 0, 0]

In [238]:
# here I try to count the number of each vocab in diff_token of each commit per developer
i=0
for word in x:
    one[vocab[word]]+=y[i]
    i+=1
    
one

['Allen Lavoie', 0.0, 0.0, 0.0, 0.0]

In [1426]:
#create a new list to repeat all above steps in a loop function
df_author_token = pd.DataFrame(columns=['Author','msg', 'tokens', 'counts'])

In [1427]:
i=0
error=0
vocab = {}
j = 1
for row in tf_source.iterrows():
    if tf_source['Commit_before'][i] is not None:
        text_before=tf_source['Commit_before'][i]
    if tf_source['Commit_after'][i] is not None:
        text_after=tf_source['Commit_after'][i]
    before = open("before-commit.py", "w+")
    after = open("after-commit.py", "w+")
    try:
        before.write(text_before)
    except ValueError:
        #print("Oops!  Error in writing to the before file #",i)
        error+=1
    try:
        after.write(text_after)
    except ValueError:
        #print("Oops!  Error in writing to the After file #",i)
        error+=1
    #this for commit before
    with tokenize.open("before-commit.py") as sf:  # need the tokenize.open for source files and not a string
        try:
            source_file_before = sf.read()
            ast_before = ast.parse(source_file_before)
        except ValueError:
            error+=1
            #print("Oops!  Error in before commit #",i)       
    bf_obj = FuncParser()
    bf_tree = ast.parse(ast_before)
    file_contents = []
    bf_obj.visit(bf_tree)
    #counting tokens in file before
    dtobj_bfore = pd.DataFrame(file_contents, columns=['token'])
    tokens_before =pd.DataFrame(dtobj_bfore['token'].value_counts())
    #doing the same for file after
    with tokenize.open("after-commit.py") as sf:  # need the tokenize.open for source files and not a string
        try:
            source_file_after = sf.read()
            ast_after = ast.parse(source_file_after)
        except ValueError:
            error+=1
            #print("Oops!  Error in after commit #",i)
    aft_obj = FuncParser()
    aft_tree = ast.parse(ast_after)
    file_contents = []
    aft_obj.visit(aft_tree)
    #counting
    dtobj_after = pd.DataFrame(file_contents, columns=['token'])
    tokens_after =pd.DataFrame(dtobj_after['token'].value_counts())
    #calculating the difference
    diff = tokens_after.subtract(tokens_before)
    diff_token = diff[(diff.select_dtypes(include=['number']) != 0).any(1)]
    diff_token=diff_token.fillna(0)
    diff_token= diff_token.abs()
    diff_token = diff_token.reset_index()
    df_author_token = df_author_token.append({'Author':tf_source['Author'][i],'msg':tf_source['msg'][i]
                                         ,'tokens':diff_token['index'],'counts':diff_token['token']},ignore_index=True)
    i+=1
    for word in diff_token['index']:
        if word in vocab:
            continue
        else:
            vocab[word]=j
            j+=1

In [1428]:
df_author_token.shape

(7039, 4)

In [1429]:
#number of vocab
j

24694

In [1430]:
import numpy as np

In [1432]:
#creating an empty matrix in size of all commits and number of tokens and try to import the number of each token per commit and developer into a new dataset
vocab_range = len(vocab)+1
cv_token = pd.DataFrame(0, index=np.arange(len(df_author_token)), columns=vocab)

In [1433]:
cv_token['commit_Author'] =0
cv_token['commit_msg']=0

In [31]:
 #vocab['orelse']

In [ ]:
x=0
for row in df_author_token.iterrows():
    y=0
    temp_token = df_author_token['tokens'][x]
    temp_count = df_author_token['counts'][x]
    for word in temp_token:
        if temp_count[y] == 0:
            cv_token[word][x]+=1
            y+=1
        else:
            cv_token[word][x]+=temp_count[y]
            y+=1
    cv_token['commit_Author'][x]=df_author_token['Author'][x]
    cv_token['commit_msg'][x]=df_author_token['msg'][x]
    x+=1
    

In [1636]:
cv_token =  pd.read_csv('tf_calls_tokens.csv')

In [1637]:
X_train_tokens  = cv_token.drop(['commit_Author','commit_msg'], axis=1)

In [1640]:
train_token_all.shape

(7039, 2062)

In [1638]:
train_token_all=X_train_tokens.loc[:, (X_train_tokens.astype(bool).sum(axis=0)>4)]

In [1641]:
test=train_token_all.columns.values

In [1624]:
test

array(['absl.testing', 'combinations', 'generate', ...,
       '_readAndCheckGraphExecutionTracesFile', '_readAndCheckGraphsFile',
       '_readAndCheckSourceFilesAndStackFrames'], dtype=object)

In [1642]:
#list_all_vocab = list(train_token_all.columns.values)

In [1643]:
tfidf_transformer = TfidfTransformer()
X_train_tfidf=tfidf_transformer.fit_transform(train_token_all)

In [1644]:
all_train_tfidf=  pd.DataFrame(X_train_tfidf.toarray(),columns=test)

In [1647]:
all_train_tfidf['commit_Author'] = cv_token['commit_Author']

In [1649]:
#all_train_tfidf

In [1136]:
all_train_tfidf[all_train_tfidf>=0.0001] = 1

In [1137]:
all_train_tfidf[all_train_tfidf<0.0001] = 0

In [1142]:
all_train_tfidf.shape

(7039, 2062)

In [1620]:
all_train_tfidf =  pd.read_csv('tf_calls_tokens.csv')

In [1650]:
all_train_tfidf_cl= all_train_tfidf.loc[:, (all_train_tfidf.astype(bool).sum(axis=0)>5)]

In [1651]:
all_train_tfidf_cl.shape

(7039, 1759)

In [352]:
train_token_cl=X_train_tokens.loc[:, (X_train_tokens.astype(bool).sum(axis=0)>5)]

In [34]:
#X_train_tokens  = cv_token.drop(['commit_Author','commit_msg'], axis=1)
from gensim.models import LogEntropyModel
from math import log, e

In [1652]:
Author_groupby=all_train_tfidf_cl.groupby(['commit_Author']).sum().reset_index()

In [385]:
from sklearn.feature_extraction.text import TfidfTransformer

In [1660]:
X_train_tokens  = Author_groupby.drop(['commit_Author'], axis=1)

In [1661]:
X_train_tokens.shape

(31, 1758)

In [1662]:
train_token_cl=X_train_tokens.loc[:, (X_train_tokens.astype(bool).sum(axis=0)>2)]

In [1663]:
train_token_cl.shape

(31, 1141)

In [1664]:
#list_cl_vocab = list(train_token_cl.columns.values)

In [1569]:
tfidf_transformer = TfidfTransformer()
X_train_tfidf=tfidf_transformer.fit_transform(train_token_cl)

In [1571]:
X_train_tfidf.mean()

0.003860732433599646

In [1572]:
dt_train_tfidf=  pd.DataFrame(X_train_tfidf.toarray(),columns=list_cl_vocab)

In [1678]:
dt_all = pd.concat([train_token_cl, dt_train_tfidf_all])

In [557]:
from mlxtend.frequent_patterns import apriori, association_rules

In [1768]:
frequent_itemsets = frequent_itemsets.reset_index()


In [104]:
import matplotlib.pyplot as plt
import matplotlib

In [450]:
def pandas_entropy(column, base):
    vc = pd.Series(column).value_counts(normalize=True, sort=False)
    return -(vc * np.log(vc)/np.log(base)).sum()

In [451]:
#model = LogEntropyModel(train_token_cl)
train_token_cl.apply(lambda x: pandas_entropy(x,2), axis=1)

0     1.347426
1     3.290522
2     1.003908
3     0.392948
4     0.478578
5     2.345980
6     1.137267
7     0.413622
8     0.404613
9     1.785008
10    5.123702
11    0.212164
12    0.490156
13    2.657800
14    1.858087
15    0.067392
16    0.190851
17    1.601309
18    0.735731
19    1.136205
20    0.792648
21    0.972891
22    0.347572
23    0.859877
24    1.741171
25    0.407716
26    0.327281
27    1.079233
28    1.023904
29    0.873709
30    1.524509
dtype: float64

In [1003]:
df = pd.DataFrame(num_cm,columns=['num-cm'])

In [1007]:
import seaborn as sns

In [112]:
All_LDA_tokens = cv_token.drop(['commit_Author', 'commit_msg'], axis=1)

In [113]:
from sklearn.decomposition import LatentDirichletAllocation
from sklearn.feature_extraction.text import TfidfTransformer

In [114]:
LDA_model_ast = LatentDirichletAllocation(n_components=5,random_state=42)

In [115]:
tfidf_transformer = TfidfTransformer()

X_train_tfidf = tfidf_transformer.fit_transform(All_LDA_tokens)
X_train_tfidf.shape

(7039, 9243)

In [116]:
LDA_model_ast.fit(All_LDA_tokens)

LatentDirichletAllocation(batch_size=128, doc_topic_prior=None,
                          evaluate_every=-1, learning_decay=0.7,
                          learning_method='batch', learning_offset=10.0,
                          max_doc_update_iter=100, max_iter=10,
                          mean_change_tol=0.001, n_components=5, n_jobs=None,
                          perp_tol=0.1, random_state=42, topic_word_prior=None,
                          total_samples=1000000.0, verbose=0)

In [117]:
test=All_LDA_tokens.columns.values
#list_vocab = list(vocab)
#len(list_vocab)

In [118]:
len(test)

9243

In [72]:
AST_domain_results = LDA_model_ast.transform(All_LDA_tokens)

In [73]:
cv_token['domain'] = AST_domain_results.argmax(axis=1)

In [87]:
AST_Author_Domain=cv_token[['commit_Author','domain']]

In [88]:
len(AST_Author_Domain)

7039

In [94]:
AST_Author_groupby=AST_Author_Domain.groupby(['domain']).size().reset_index(name='count')

In [101]:
topic_per=((AST_Author_groupby)/len(AST_Author_Domain))*100